# 02Softmax Regression

In [1]:
# import needed package
%matplotlib inline
from IPython import display
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import time

import sys
import utils
print(torch.__version__)
print(torchvision.__version__)

1.2.0
0.4.0


### 获取和读取数据

In [2]:
import os
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))
print(BASE_DIR)

D:\Project\Anaconda


In [3]:
batch_size = 256
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size, root=os.path.join(BASE_DIR,'dataset'))

0it [00:00, ?it/s]

 97%|███████████████████████████████████████████████████████████████▏ | 25698304/26421880 [00:20<00:00, 1916280.51it/s]

Extracting D:\Project\Anaconda\dataset\FashionMNIST\raw\train-images-idx3-ubyte.gz to D:\Project\Anaconda\dataset\FashionMNIST\raw



0it [00:00, ?it/s]


  0%|                                                                                        | 0/29515 [00:02<?, ?it/s]
32768it [00:02, 11024.54it/s]                                                                                          

0it [00:00, ?it/s]

...


100%|███████████████████████████████████████████████████████████████████▉| 4415488/4422102 [00:11<00:00, 680662.48it/s]

0it [00:00, ?it/s]

Extracting D:\Project\Anaconda\dataset\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to D:\Project\Anaconda\dataset\FashionMNIST\raw




8192it [00:03, 2108.81it/s]                                                                                            

Extracting D:\Project\Anaconda\dataset\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to D:\Project\Anaconda\dataset\FashionMNIST\raw
Processing...
Done!



26427392it [00:40, 1916280.51it/s]                                                                                     
4423680it [00:26, 680662.48it/s]                                                                                       

### 初始化模型参数

In [ ]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)

In [ ]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

### softmax
$$\hat y_j = \frac{exp(o_j)}{\sum\limits_{i=1}^3 exp(o_i)}$$

In [ ]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition

In [ ]:
X = torch.rand((2, 5))
X_prob = softmax(X)
print(X_prob, X_prob.sum(dim=1))

### 定义模型

In [ ]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)

### 定义损失函数
$$Loss = - \sum\limits_{j=1}^q y_j^{i}log\hat y_j^(i)$$
$$Cost = - \frac{1}{n}\sum\limits_{j=1}^q y_j^{i}log\hat y_j^(i) $$

In [ ]:
y_pred = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_pred.gather(1, y.view(-1, 1)) # -1 表示不确定

In [ ]:
def cross_entropy(y_pred, y):
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))

### 定义准确率accuracy

In [ ]:
def accuracy(y_pred, y):
    return (y_pred.argmax(dim=1) == y).float().mean().item()

In [ ]:
print(accuracy(y_pred, y))

In [ ]:
num_epochs, lr =5, 0.1

def train_softmax(net, train_iter, test_iter, loss, num_epochs, batch_size, params=None,
                 lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_pred = net(X)
            l = loss(y_pred, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                utils.sgd(params, lr, batch_size)
            else:
                optimizer.step()
                
            train_l_sum += l.item()
            train_acc_sum += (y_pred.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
             %(epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))
train_softmax(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)